# 1. 문장 2207건 ChatGPT 입력용 데이터 만들기

In [1]:
import pandas as pd
from tqdm import tqdm
import os, sys
sys.path.append(os.path.abspath(".."))
from Read_Data import ReadKiprisData
from transformers import ElectraModel, ElectraTokenizer
from typing import Dict, Tuple
import re
import openai
from openai import OpenAI

/home/hee/anaconda3/envs/korpatE/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
# path 설정
patent_data_path = "../data/ner_dic_ami"
save_path = "../data/output"

In [12]:
read_data = ReadKiprisData()
data_df = read_data.run(patent_data_path)

  0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:00<00:00,  7.44it/s]


In [13]:
# 모델 경로 (KIPI-KorPatELECTRA 경로 설정)
model_path = "../KIPI-KorPatELECTRA/KorPatELECTRA/PT"

# 토크나이저 및 모델 불러오기
tokenizer = ElectraTokenizer.from_pretrained(model_path)
model = ElectraModel.from_pretrained(model_path)

Some weights of the model checkpoint at ../KIPI-KorPatELECTRA/KorPatELECTRA/PT were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
# 1. 데이터 전처리
# ex) "1."" -> "제 1항"
def text_replace(text: str)-> str:
    new_text = re.sub(r"(\d)\.", lambda match: f"제 {match.group(1)} 항.", text)
    return new_text

sen_df = pd.DataFrame({'문장': list(data_df['문장'].unique())})
sen_df['문장'] = sen_df['문장'].apply(lambda sen: text_replace(sen))
sen_df.head()

,문장
0,"제 1 항. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 ..."
1,"제 2 항. 단말기에 콘텐트 리스트를 전송하는 콘텐트 리스트 제공부와,분산처리 작업..."
2,"제 3 항. 제 2 항에 있어서, 상기 스트리밍 처리부는 상기 단말기로부터 선택된 ..."
3,"제 4 항. 제 3 항에 있어서, 상기 스트리밍 처리부는 상기 콘텐트 목록에 속한 ..."
4,"제 5 항. 제 4 항에 있어서, 상기 작업량 계산부는 상기 콘텐트의 메타 정보에 ..."


In [67]:
# 2. 확장
def tokenizing_result(sen: str)-> Tuple[list, list]:
    tokens = tokenizer.tokenize(sen)
    tokens = ["[CLS]"] + tokens + ["[SEP]"]
    input_ids = tokenizer(sen)['input_ids']

    return tokens, input_ids

tqdm.pandas(desc="문장 토크나이징")

sen_df[["tokens", "input_ids"]] = sen_df["sen"].progress_apply(
    lambda sentence: pd.Series(tokenizing_result(sentence)))

expanded_df = sen_df.explode(["tokens", "input_ids"]).reset_index(drop=True) # 리스트를 행으로 확장
expanded_df

문장 토크나이징: 100%|██████████| 2207/2207 [00:03<00:00, 637.31it/s]


,sen,tokens,input_ids
0,"제 1 항. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 ...",[CLS],2
1,"제 1 항. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 ...",제,1749
2,"제 1 항. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 ...",1,10
3,"제 1 항. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 ...",항,2390
4,"제 1 항. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 ...",.,8
...,...,...,...
141041,상기 확인부에서 확인된 사용자의 신분을 공개하는 신분 공개부를 더 포함함을 특징으로...,이동통신단말기,10854
141042,상기 확인부에서 확인된 사용자의 신분을 공개하는 신분 공개부를 더 포함함을 특징으로...,##의,2715
141043,상기 확인부에서 확인된 사용자의 신분을 공개하는 신분 공개부를 더 포함함을 특징으로...,인증,5276
141044,상기 확인부에서 확인된 사용자의 신분을 공개하는 신분 공개부를 더 포함함을 특징으로...,##시스템,5671


In [ ]:
# expanded_df.to_csv(os.path.join(save_path, "patentData_token_ids_2207.csv"), encoding='euc-kr')

In [90]:
def tokenizing_result(sen: str)-> Tuple[list, list]:
    tokens = tokenizer.tokenize(sen)
    tokens = ["[CLS]"] + tokens + ["[SEP]"]

    return tokens

tqdm.pandas(desc="문장 토크나이징")

sen_df["tokens"] = sen_df["sen"].progress_apply(
    lambda sentence:tokenizing_result(sentence))

expanded_df2 = sen_df.explode("tokens").reset_index(drop=True) # 리스트를 행으로 확장
expanded_df2

문장 토크나이징: 100%|██████████| 2207/2207 [00:01<00:00, 1757.56it/s]


,sen,tokens
0,"제 1 항. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 ...",[CLS]
1,"제 1 항. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 ...",제
2,"제 1 항. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 ...",1
3,"제 1 항. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 ...",항
4,"제 1 항. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 ...",.
...,...,...
141041,상기 확인부에서 확인된 사용자의 신분을 공개하는 신분 공개부를 더 포함함을 특징으로...,이동통신단말기
141042,상기 확인부에서 확인된 사용자의 신분을 공개하는 신분 공개부를 더 포함함을 특징으로...,##의
141043,상기 확인부에서 확인된 사용자의 신분을 공개하는 신분 공개부를 더 포함함을 특징으로...,인증
141044,상기 확인부에서 확인된 사용자의 신분을 공개하는 신분 공개부를 더 포함함을 특징으로...,##시스템


In [93]:
# 문장 번호 추가 
expanded_df2['sen_idx'] = expanded_df2.groupby('sen').ngroup()
expanded_df2

,sen,tokens,sen_idx
0,"제 1 항. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 ...",[CLS],1402
1,"제 1 항. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 ...",제,1402
2,"제 1 항. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 ...",1,1402
3,"제 1 항. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 ...",항,1402
4,"제 1 항. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 ...",.,1402
...,...,...,...
141041,상기 확인부에서 확인된 사용자의 신분을 공개하는 신분 공개부를 더 포함함을 특징으로...,이동통신단말기,1293
141042,상기 확인부에서 확인된 사용자의 신분을 공개하는 신분 공개부를 더 포함함을 특징으로...,##의,1293
141043,상기 확인부에서 확인된 사용자의 신분을 공개하는 신분 공개부를 더 포함함을 특징으로...,인증,1293
141044,상기 확인부에서 확인된 사용자의 신분을 공개하는 신분 공개부를 더 포함함을 특징으로...,##시스템,1293


In [16]:
def tokenizing_result(sen: str)-> Tuple[list, list]:
    tokens = tokenizer.tokenize(sen)
    tokens = ["[CLS]"] + tokens + ["[SEP]"]

    return ", ".join(token for token in tokens)

sen_df['토큰'] = sen_df['문장'].apply(lambda data: tokenizing_result(data))
sen_df

,문장,토큰
0,"제 1 항. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 ...","[CLS], 제, 1, 항, ., 분산, ##처리, 작업, 데이터, ##를, 처리,..."
1,"제 2 항. 단말기에 콘텐트 리스트를 전송하는 콘텐트 리스트 제공부와,분산처리 작업...","[CLS], 제, 2, 항, ., 단말기, ##에, 콘텐트, 리스트, ##를, 전송..."
2,"제 3 항. 제 2 항에 있어서, 상기 스트리밍 처리부는 상기 단말기로부터 선택된 ...","[CLS], 제, 3, 항, ., 제, 2, 항, ##에, 있, ##어서, ,, 상..."
3,"제 4 항. 제 3 항에 있어서, 상기 스트리밍 처리부는 상기 콘텐트 목록에 속한 ...","[CLS], 제, 4, 항, ., 제, 3, 항, ##에, 있, ##어서, ,, 상..."
4,"제 5 항. 제 4 항에 있어서, 상기 작업량 계산부는 상기 콘텐트의 메타 정보에 ...","[CLS], 제, 5, 항, ., 제, 4, 항, ##에, 있, ##어서, ,, 상..."
...,...,...
2202,상기 그룹 비밀 키 발생부의 그룹 비밀 키로부터 이동통신단말기가 인증을 위해 그룹 ...,"[CLS], 상기, 그룹, 비밀, 키, 발생부, ##의, 그룹, 비밀, 키로, ##..."
2203,상기 홈 네트워크 상의 가전기기(ex. 디지털TV...)를 원격제어 하기 위해 이동...,"[CLS], 상기, 홈, 네트워크, 상의, 가전기기, (, ex, ., 디지털, #..."
2204,"제 5 항. 제 4항에 있어서, 상기 홈 네트워크의 사용 중 문제가 발생 시 상기 ...","[CLS], 제, 5, 항, ., 제, 4, ##항, ##에, 있, ##어서, ,,..."
2205,상기 그룹 공개 키 수집부에서 수집된 그룹 공개 키에서 각각의 서명을 확인하여 분실...,"[CLS], 상기, 그룹, 공개, 키, 수집부, ##에, ##서, 수집, ##된, ..."


In [17]:
sen_df.to_csv(os.path.join(save_path, "patentData_2207_v3.csv"), encoding='euc-kr')

In [97]:
# expanded_df2.to_csv("patentData_token_idx_2207.csv", encoding='euc-kr')

## 2. ChatGPT API 쓰기

In [7]:
!pip install openai

In [20]:
def tokenizing_result(sen: str)-> Tuple[list, list]:
    tokens = tokenizer.tokenize(sen)
    tokens = ["[CLS]"] + tokens + ["[SEP]"]

    return ", ".join(token for token in tokens)

def create_query(sen: str, tokens: str)-> str:
    base_query = "특허 전문가로서 특허 청구항 일부인 아래 문장과 토큰을 보고, 특허 도메인에서 유의미한 값을 갖는 개체명을 찾아줘. 유의미한 개체명이 없는 경우 <None>이라고 답변해줘."
    result_query = f'{base_query}\n문장:{sen}\n토큰:{tokens}'
    return result_query

sen_df2 = sen_df.copy()
sen_df2['token'] = None; sen_df2['entity'] = None

for idx, row in sen_df.iterrows():
    sentence=row['sen']
    tokens = tokenizing_result(sentence)
    sen_df2.loc[idx, 'tokens'] = ", ".join(token for token in tokens)
    query = create_query(sentence, tokens)

    break
print(query)

특허 전문가로서 특허 청구항 일부인 아래 문장과 토큰을 보고, 특허 도메인에서 유의미한 값을 갖는 개체명을 찾아줘. 유의미한 개체명이 없는 경우 <None>이라고 답변해줘.
문장:제 1 항. 분산처리 작업 데이터를 처리하는 단말기와,상기 분산처리 작업 데이터를 상기 단말기에 할당하여 분산처리 작업을 요청하는 분산처리 작업관리 장치를 포함하며,상기 분산처리 작업관리 장치는, 상기 분산처리 작업의 대가로 상기 단말기에 제공되는 콘텐트의 재생시간 및 상기 단말기에서 상기 콘텐트의 재생에 필요한 자원량을 기초로 계산된 상기 단말기의 유휴 자원량을 이용하여, 상기 단말기의 분산처리 가능 작업량을 계산하고, 상기 계산된 분산처리 가능 작업량에 따른 분산처리 작업 데이터를 상기 단말기에 할당하는 것을 특징으로 하는 분산 컴퓨팅 제공 시스템
토큰:[CLS], 제, 1, 항, ., 분산, ##처리, 작업, 데이터, ##를, 처리, ##하, ##는, 단말기, ##와, ,, 상기, 분산, ##처리, 작업, 데이터, ##를, 상기, 단말기, ##에, 할당, ##하여, 분산, ##처리, 작업, ##을, 요청, ##하, ##는, 분산, ##처리, 작업, ##관리, 장치, ##를, 포함, ##하, ##며, ,, 상기, 분산, ##처리, 작업, ##관리, 장치, ##는, ,, 상기, 분산, ##처리, 작업, ##의, 대가, ##로, 상기, 단말기, ##에, 제공, ##되, ##는, 콘텐트, ##의, 재생시, ##간, 및, 상기, 단말기, ##에, ##서, 상기, 콘텐트, ##의, 재생, ##에, 필요, ##한, 자원, ##량, ##을, 기초, ##로, 계산, ##된, 상기, 단말기, ##의, 유휴, 자원, ##량, ##을, 이용, ##하여, ,, 상기, 단말기, ##의, 분산, ##처리, 가능, 작업, ##량, ##을, 계산, ##하, ##고, ,, 상기, 계산, ##된, 분산, ##처리, 가능, 작업, ##량, ##에, 따른, 분산, ##처리, 작업, 데이터, ##를, 상기, 단말기, 

In [7]:
from dotenv import load_dotenv
import os, openai
load_dotenv()
api_key = os.getenv("GPT_API_KEY")

In [9]:
openai.api_key = api_key

model = 'gpt-4o'
query = '''너는 대한민국에서 가장 유능한 특허 전문가야. 특허 전문가로서 특허 청구항 일부인 아래 문장과 토큰을 보고, 특허 도메인에서 유의미한 값을 갖는 개체명을 찾아줘. 유의미한 개체명이 없는 경우 <None>이라고 답변해줘. 출력은 설명없이 개체명만 ", "로 이어서 출력해줘.
문장 : 제 1항에 있어서,상기 시스템은,상기 촬영이미지에 대한 가이드이미지를 생성하는 가이드생성부 및,상기 가이드이미지를 상기 유저에게 제공하는 가이드제공부를 포함하는 가이드제공모듈;을 더 포함하며,상기 질문입력모듈은,상기 가이드이미지를 제공받은 상기 유저로부터 상기 유저의 안면부를 촬영한 적어도 하나 이상의 촬영이미지 및 관련질문을 입력받는 것을 특징으로 하는, 실시간 메이크업 진단 및 질의응답 서비스 제공 시스템.	
토큰 : [CLS], 제, 1, ##항, ##에, 있, ##어서, ,, 상기, 시스템, ##은, ,, 상기, 촬영, ##이미지, ##에, 대한, 가이드, ##이미지, ##를, 생성, ##하, ##는, 가이드, ##생성부, 및, ,, 상기, 가이드, ##이미지, ##를, 상기, 유저, ##에, ##게, 제공, ##하, ##는, 가이드, ##제공부, ##를, 포함, ##하, ##는, 가이드, ##제공, ##모듈, [UNK], 을, 더, 포함, ##하, ##며, ,, 상기, 질문, ##입력, ##모듈, ##은, ,, 상기, 가이드, ##이미지, ##를, 제공, ##받, ##은, 상기, 유저, ##로, ##부터, 상기, 유저, ##의, 안면, ##부, ##를, 촬영, ##한, 적어도, 하나, 이상, ##의, 촬영, ##이미지, 및, 관련, ##질, ##문, ##을, 입력, ##받, ##는, 것, ##을, 특징, ##으로, 하, ##는, ,, 실시간, 메이크업, 진단, 및, 질의, ##응답, 서비스, 제공, 시스템, ., [SEP]
'''

# 메시지 설정하기
messages = [{
    "role": "system",
    "content": "You are the nation's top patent expert."
}, {
    "role": "user",
    "content": query
}]

response = openai.chat.completions.create(
    messages=messages,
    model=model,
    temperature=0.1  
)
response

ChatCompletion(id='chatcmpl-AreTpztN6G6ZdTl4bAxYFtlx69n9u', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='가이드이미지, 가이드생성부, 가이드제공부, 가이드제공모듈, 질문입력모듈, 실시간 메이크업 진단 및 질의응답 서비스 제공 시스템', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1737350353, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_50cad350e4', usage=CompletionUsage(completion_tokens=48, prompt_tokens=568, total_tokens=616, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [10]:
answer = response.choices[0].message.content
print(answer)

가이드이미지, 가이드생성부, 가이드제공부, 가이드제공모듈, 질문입력모듈, 실시간 메이크업 진단 및 질의응답 서비스 제공 시스템
